In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from broinsight.data_quality.sql_profile import sql_table_profile, sql_field_profile

In [3]:
import seaborn as sns

df = sns.load_dataset('tips')

In [13]:
from broinsight.data_quality.field_profile import field_profile
from broinsight.data_quality.table_profile import table_profile
_ = field_profile(df)
_

{'total_bill': {'data_types': 'float',
  'missing_values': 0,
  'missing_values_pct': 0.0,
  'unique_values': 229,
  'unique_values_pct': 0.94,
  'most_frequent': {13.42: 3, 13.81: 2, 15.98: 2, 17.92: 2, 10.07: 2},
  'statistics': {'min': 3.07,
   'max': 50.81,
   'mean': 19.79,
   'median': 17.8,
   'std': 8.9,
   'var': 79.25,
   'skew': 1.13,
   'kurt': 1.22,
   'iqr': 10.78,
   'cv': 0.45,
   'lower_bound': -2.82,
   'upper_bound': 40.3}},
 'tip': {'data_types': 'float',
  'missing_values': 0,
  'missing_values_pct': 0.0,
  'unique_values': 123,
  'unique_values_pct': 0.5,
  'most_frequent': {2.0: 33, 3.0: 23, 4.0: 12, 5.0: 10, 2.5: 10},
  'statistics': {'min': 1.0,
   'max': 10.0,
   'mean': 3.0,
   'median': 2.9,
   'std': 1.38,
   'var': 1.91,
   'skew': 1.47,
   'kurt': 3.65,
   'iqr': 1.56,
   'cv': 0.46,
   'lower_bound': -0.34,
   'upper_bound': 5.91}},
 'sex': {'data_types': 'string',
  'missing_values': 0,
  'missing_values_pct': 0.0,
  'unique_values': 2,
  'unique_values

In [70]:
import pandas as pd
profile = pd.DataFrame.from_dict(_, orient="index")
print(profile.to_string())

           data_types  missing_values  missing_values_pct  unique_values  unique_values_pct                                       most_frequent                                                                                                                                                                             statistics
total_bill      float               0                 0.0            229               0.94  {13.42: 3, 13.81: 2, 15.98: 2, 17.92: 2, 10.07: 2}  {'min': 3.07, 'max': 50.81, 'mean': 19.79, 'median': 17.8, 'std': 8.9, 'var': 79.25, 'skew': 1.13, 'kurt': 1.22, 'iqr': 10.78, 'cv': 0.45, 'lower_bound': -2.82, 'upper_bound': 40.3}
tip             float               0                 0.0            123               0.50       {2.0: 33, 3.0: 23, 4.0: 12, 5.0: 10, 2.5: 10}        {'min': 1.0, 'max': 10.0, 'mean': 3.0, 'median': 2.9, 'std': 1.38, 'var': 1.91, 'skew': 1.47, 'kurt': 3.65, 'iqr': 1.56, 'cv': 0.46, 'lower_bound': -0.34, 'upper_bound': 5.91}
sex            stri

In [71]:
from broinsight.data_quality.criteria import assess_data_quality

dq_summary = assess_data_quality(_)

In [72]:
dq_summary

{'total_bill': {'summary': {'quality': 'good',
   'issues': [{'type': 'moderate_skewness',
     'severity': 'minor',
     'description': 'Moderately skewed distribution (skew: 1.13)'}],
   'issue_count': 1}},
 'tip': {'summary': {'quality': 'good',
   'issues': [{'type': 'moderate_skewness',
     'severity': 'minor',
     'description': 'Moderately skewed distribution (skew: 1.47)'}],
   'issue_count': 1}},
 'sex': {'summary': {'quality': 'good', 'issues': [], 'issue_count': 0}},
 'smoker': {'summary': {'quality': 'good', 'issues': [], 'issue_count': 0}},
 'day': {'summary': {'quality': 'good', 'issues': [], 'issue_count': 0}},
 'time': {'summary': {'quality': 'good', 'issues': [], 'issue_count': 0}},
 'size': {'summary': {'quality': 'good',
   'issues': [{'type': 'moderate_skewness',
     'severity': 'minor',
     'description': 'Moderately skewed distribution (skew: 1.45)'}],
   'issue_count': 1}}}

In [73]:
from broinsight.data_quality.create_profile import create_profile, format_profile

profile_dict = create_profile(df)
profile_str = format_profile(profile_dict)

In [74]:
profile_dict

{'dataset_summary': {'rows': 244,
  'columns': 7,
  'duplicates': 1,
  'duplicate_examples': {198: {'total_bill': 13.0,
    'tip': 2.0,
    'sex': 'Female',
    'smoker': 'Yes',
    'day': 'Thur',
    'time': 'Lunch',
    'size': 2},
   202: {'total_bill': 13.0,
    'tip': 2.0,
    'sex': 'Female',
    'smoker': 'Yes',
    'day': 'Thur',
    'time': 'Lunch',
    'size': 2}}},
 'fields': {'total_bill': {'profile': {'data_types': 'float',
    'missing_values': 0,
    'missing_values_pct': 0.0,
    'unique_values': 229,
    'unique_values_pct': 0.94,
    'most_frequent': {13.42: 3, 13.81: 2, 15.98: 2, 17.92: 2, 10.07: 2},
    'statistics': {'min': 3.07,
     'max': 50.81,
     'mean': 19.79,
     'median': 17.8,
     'std': 8.9,
     'var': 79.25,
     'skew': 1.13,
     'kurt': 1.22,
     'iqr': 10.78,
     'cv': 0.45,
     'lower_bound': -2.82,
     'upper_bound': 40.3}},
   'quality': {'quality': 'good',
    'issues': [{'type': 'moderate_skewness',
      'severity': 'minor',
      'des

In [75]:
print(profile_str)

# Dataset Overview
**Size:** 244 rows × 7 columns
**Duplicates:** 1 duplicate record(s) found
**Examples:** Rows 198, 202 are identical

# Fields
## total_bill
**Type:** float
**Missing:** 0 (0.0%)
**Unique:** 229 (94.0%)
**Quality:** good
**Issues:**
  - Moderately skewed distribution (skew: 1.13) (minor)
**Stats:** min=3.07, max=50.81, mean=19.79, skew=1.13

## tip
**Type:** float
**Missing:** 0 (0.0%)
**Unique:** 123 (50.0%)
**Quality:** good
**Issues:**
  - Moderately skewed distribution (skew: 1.47) (minor)
**Stats:** min=1.0, max=10.0, mean=3.0, skew=1.47

## sex
**Type:** string
**Missing:** 0 (0.0%)
**Unique:** 2 (1.0%)
**Quality:** good
**Stats:** mode=Male, avg_length=4.71

## smoker
**Type:** string
**Missing:** 0 (0.0%)
**Unique:** 2 (1.0%)
**Quality:** good
**Stats:** mode=No, avg_length=2.38

## day
**Type:** string
**Missing:** 0 (0.0%)
**Unique:** 4 (2.0%)
**Quality:** good
**Stats:** mode=Sat, avg_length=3.25

## time
**Type:** string
**Missing:** 0 (0.0%)
**Unique:** 

In [76]:
from broinsight.experiment.ollama import LocalOpenAI
from broprompt import Prompt

model = LocalOpenAI()

prompt = Prompt.from_markdown("broinsight/prompt_hub/dq_suggestion.md")
question = "Do we have any concern about this one?"
user_input = "PROFILE:\n\n{profile}\n\nUSER_INPUT:\n\n{question}\n\n".format(profile=profile_str, question=question)
response = model.run(system_prompt=prompt.str, messages=[
    model.UserMessage(text=user_input)
])
response

{'content': '**Data Quality Assessment:** READY  \n\n**Critical Issues Found:** None – all fields are complete and correctly typed.  \n\n**Recommended Actions (prioritized):**  \n1. **Duplicate record** – Rows 198 and 202 are identical.  \n   *Action:* Decide whether to keep or drop the duplicate based on domain knowledge (e.g., if the duplicate represents a genuine repeated visit, keep it; otherwise, remove it).  \n2. **Moderate skewness** in `total_bill`, `tip`, and `size` (skew\u202f≈\u202f1.1‑1.5).  \n   *Action:*  \n   - If using algorithms that assume normality (e.g., linear regression, ANOVA), consider a log or square‑root transformation.  \n   - For tree‑based models (random forest, gradient boosting) the skewness is usually not a problem.  \n3. **No missing values** – no imputation needed.  \n\n**Next Steps:**  \n- Proceed with exploratory data analysis (EDA).  \n- If you plan to build predictive models, evaluate whether transformations improve model assumptions or performance

In [77]:
print(response['content'])

**Data Quality Assessment:** READY  

**Critical Issues Found:** None – all fields are complete and correctly typed.  

**Recommended Actions (prioritized):**  
1. **Duplicate record** – Rows 198 and 202 are identical.  
   *Action:* Decide whether to keep or drop the duplicate based on domain knowledge (e.g., if the duplicate represents a genuine repeated visit, keep it; otherwise, remove it).  
2. **Moderate skewness** in `total_bill`, `tip`, and `size` (skew ≈ 1.1‑1.5).  
   *Action:*  
   - If using algorithms that assume normality (e.g., linear regression, ANOVA), consider a log or square‑root transformation.  
   - For tree‑based models (random forest, gradient boosting) the skewness is usually not a problem.  
3. **No missing values** – no imputation needed.  

**Next Steps:**  
- Proceed with exploratory data analysis (EDA).  
- If you plan to build predictive models, evaluate whether transformations improve model assumptions or performance.  
- Document the duplicate handling 

In [4]:
import duckdb

conn = duckdb.connect()

conn.register("tips", df)

In [5]:
# More detailed schema information
result = conn.execute("""
    SELECT column_name, data_type, is_nullable
    FROM information_schema.columns 
    WHERE table_name = 'tips'
""").fetchdf()
print(result)

  column_name                          data_type is_nullable
0  total_bill                             DOUBLE         YES
1         tip                             DOUBLE         YES
2         sex             ENUM('Male', 'Female')         YES
3      smoker                  ENUM('Yes', 'No')         YES
4         day  ENUM('Thur', 'Fri', 'Sat', 'Sun')         YES
5        time            ENUM('Lunch', 'Dinner')         YES
6        size                             BIGINT         YES


In [14]:
table_profile(df)

{'rows': 244,
 'columns': 7,
 'duplicates': 1,
 'evidences': {198: {'total_bill': 13.0,
   'tip': 2.0,
   'sex': 'Female',
   'smoker': 'Yes',
   'day': 'Thur',
   'time': 'Lunch',
   'size': 2},
  202: {'total_bill': 13.0,
   'tip': 2.0,
   'sex': 'Female',
   'smoker': 'Yes',
   'day': 'Thur',
   'time': 'Lunch',
   'size': 2}}}

In [6]:
sql_table_profile(conn, "tips")

{'rows': 244,
 'columns': 7,
 'duplicates': 1,
 'evidences': {0: {'total_bill': 13.0,
   'tip': 2.0,
   'sex': 'Female',
   'smoker': 'Yes',
   'day': 'Thur',
   'time': 'Lunch',
   'size': 2,
   'dup_count': 2}}}

In [16]:
f_p = field_profile(df)

In [17]:
s_p = sql_field_profile(conn, "tips")

In [10]:
conn.execute("DESCRIBE tips").fetchall()

[('total_bill', 'DOUBLE', 'YES', None, None, None),
 ('tip', 'DOUBLE', 'YES', None, None, None),
 ('sex', "ENUM('Male', 'Female')", 'YES', None, None, None),
 ('smoker', "ENUM('Yes', 'No')", 'YES', None, None, None),
 ('day', "ENUM('Thur', 'Fri', 'Sat', 'Sun')", 'YES', None, None, None),
 ('time', "ENUM('Lunch', 'Dinner')", 'YES', None, None, None),
 ('size', 'BIGINT', 'YES', None, None, None)]

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [19]:
f_p.keys()

dict_keys(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'])

In [20]:
s_p.keys()

dict_keys(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'])

In [23]:
field = 'sex'
f_p[field]

{'data_types': 'string',
 'missing_values': 0,
 'missing_values_pct': 0.0,
 'unique_values': 2,
 'unique_values_pct': 0.01,
 'most_frequent': {'Male': 157, 'Female': 87},
 'statistics': {'mode': 'Male',
  'avg_length': 4.71,
  'min_length': 4.0,
  'max_length': 6.0,
  'empty_count': 0.0,
  'whitespace_count': 0.0,
  'pattern_consistency': 0.01}}

In [24]:
s_p[field]

{'data_types': 'string',
 'missing_values': 0,
 'missing_values_pct': 0.0,
 'unique_values': 2,
 'unique_values_pct': 0.01,
 'most_frequent': {'Male': 157, 'Female': 87},
 'statistics': {'mode': 'Male',
  'avg_length': 4.71,
  'min_length': 4,
  'max_length': 6,
  'empty_count': 0,
  'whitespace_count': 0,
  'pattern_consistency': 0.01}}

In [25]:
from broinsight.data_quality.criteria import assess_data_quality
assess_data_quality(s_p)

{'total_bill': {'summary': {'quality': 'good',
   'issues': [{'type': 'moderate_skewness',
     'severity': 'minor',
     'description': 'Moderately skewed distribution (skew: 1.13)'}],
   'issue_count': 1}},
 'tip': {'summary': {'quality': 'good',
   'issues': [{'type': 'moderate_skewness',
     'severity': 'minor',
     'description': 'Moderately skewed distribution (skew: 1.47)'}],
   'issue_count': 1}},
 'sex': {'summary': {'quality': 'good', 'issues': [], 'issue_count': 0}},
 'smoker': {'summary': {'quality': 'good', 'issues': [], 'issue_count': 0}},
 'day': {'summary': {'quality': 'good', 'issues': [], 'issue_count': 0}},
 'time': {'summary': {'quality': 'good', 'issues': [], 'issue_count': 0}},
 'size': {'summary': {'quality': 'good',
   'issues': [{'type': 'moderate_skewness',
     'severity': 'minor',
     'description': 'Moderately skewed distribution (skew: 1.45)'}],
   'issue_count': 1}}}